# Part 0: Dataloader and Visualizations

In [1]:
import numpy as np
from PIL import Image
import scipy.io
import torch

import torch.nn as nn
import torch.nn.functional as F

import torchvision.models as models
from torchvision import transforms, datasets

from torch.utils.data import DataLoader
import wandb

from voc_dataset import VOCDataset
from utils import *

USE_WANDB = False

## Q0.1: Editing the Dataloader
The first part of the assignment involves editing the dataloader so that we can access bounding-box proposals as well as the ground-truth bounding boxes. The ground truth bounding box can be accessed through the VOC Dataset annotations itself and we have completed this part for you in the starter code. 

Unsupervised bounding box proposals are obtained through methods such as [Selective Search](https://ivi.fnwi.uva.nl/isis/publications/2013/UijlingsIJCV2013/UijlingsIJCV2013.pdf). Since Selective Search is slow to run on each image, we have pre-computed the bounding box proposals for you (you downloaded this in the data preparation step).

Your task is to change the dataloader to obtain the proposed bounding boxes for each image. Feel free to experiment with the data in the files to figure out the number of proposals per image, their scores, etc. Returning a dictionary would be convenient here. For the bounding boxes, using the relative positions is usually a better idea since they are invariant to changes in the size of the image.

In [2]:
dataset = VOCDataset('trainval', top_n=10)

**Q0.1**: Load the image corresponding to index 2020 and print the GT labels associated with it.

**Hint**: items at a particular index can be accesed by usual indexing notation (dataset[idx])

In [20]:
# TODO: get the image information from index 2020
idx = 2020
labels = dataset[idx]["label"]
for i, label in enumerate(labels):
    if label>0:
        print(VOCDataset.get_class_name(i))

# p = dataset[idx]["rois"]
# print(p.keys())
# print(p["boxScores"].flatten().shape)
# print(p["boxScores"][0].shape)
# score = p["boxScores"][0][0].flatten()
# print(score)
# (-score).argsort()[:10]
# print(p.keys())
# print(p["boxes"][0][1][0]/2)
# proposals = dataset[idx]["rois"]
# print(proposals)
# print(p["boxScores"].flatten()[1])

TypeError: 'VOCDataset' object is not callable

## Q0.2 and Q0.3: Wandb Logging
First, let's initialize a Weights and Biases project. 

In [5]:
USE_WANDB = True
if USE_WANDB:
    wandb.init(project="vlr-hw1", reinit=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: joshuarain (use `wandb login --relogin` to force relogin)
D:\anaconda\envs\vlr\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


**Q0.2**: Complete this block for overlaying the ground truth box on an image.

**Hint**: convert the image tensor to a PIL image and plot it (check `utils.py` for helper functions). You can use [this](https://docs.wandb.ai/library/log) as a reference for logging syntax.

In [6]:
class_id_to_label = dict(enumerate(dataset.CLASS_NAMES))

# TODO: load the GT information corresponding to index 2020.
original_image = tensor_to_PIL(dataset[idx]["image"])
gt_labels = dataset[idx]["gt_classes"]
gt_boxes = dataset[idx]["gt_boxes"]

img = wandb.Image(original_image, boxes={
    "predictions": {
        "box_data": get_box_data(gt_labels, gt_boxes),
        "class_labels": class_id_to_label,
    },
})

# TODO: log the GT bounding box
wandb.log({"GT bounding box": img})

**Q0.3**: Visualize the top 10 bounding proposals corresponding to index 2020.

**Hint**: Check the `get_box_data` function in `utils.py` and understand how it is being used. 

In [19]:
# TODO: plot top ten proposals (of bounding boxes)
proposals = dataset[idx]["rois"]
proposal_labels = []
for i, proposal in enumerate(proposals):
    labels = [gt_labels[i] for _ in proposal]
    proposal_labels.append(labels)

print(gt_labels)
img = wandb.Image(original_image, boxes={
    "predictions": {
        "box_data": get_box_data(proposal_labels, proposals),
        "class_labels": class_id_to_label,
    },
})

# TODO: log the GT bounding box
# wandb.log({"10 proposal bboxes": img})

IndexError: list index out of range

In [9]:
labels = ["A" for _ in range(10)]
print(labels)

['A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A']
